# Cryptographic protocol demonstration
## Installing libraries
pip install pycryptodome
pip install cryptography

In [94]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.backends import default_backend

from Crypto.Cipher import PKCS1_OAEP

import time
import secrets
import hashlib
from datetime import datetime

In [95]:
message = "The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks"

In [96]:
# Creates a timestamp using the current time in seconds in Unix format
timestamp = int(time.time())
print(f"Timestamp: {timestamp}")

Timestamp: 1710087696


In [97]:
# Generate a nonce as a 32-character hex string
nonce_hex = secrets.token_hex(16)  
print(f"Nonce (hex): {nonce_hex}")

nonce_storage = []

Nonce (hex): afcd8d0d6d387866879d303001815b74


In [98]:
# Concatenate with a delimiter (:)
combined = f"{nonce_hex}:{timestamp}:{message}"
print("Combined string:", combined)

Combined string: afcd8d0d6d387866879d303001815b74:1710087696:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


In [99]:
# Encode the combined string to bytes, then hash it using SHA-256
hashed = hashlib.sha256(combined.encode()).hexdigest()

print("SHA-256 Hash:", hashed)

SHA-256 Hash: 50d9203948a8847d90f9aa25476d11b258f5b6eeef3af4385fd6f5cf6e625573


In [100]:
def generate_rsa_keys():
    """
    Generates an RSA private and public key pair.
    
    The public_exponent is set to 65537, which is a common choice for RSA encryption.
    It's a prime number that ensures efficiency and security in the encryption process.
    The key size of 2048 bits is selected to provide a good balance between security and performance,
    making it sufficiently secure for most applications.
    
    Returns:
        A tuple containing the RSA private key and public key.
    """
    # Generate the private key with specified parameters
    private_key = rsa.generate_private_key(
        public_exponent=65537,  # Commonly used public exponent
        key_size=2048,         # Key size of 2048 bits
        backend=default_backend()  # Use the default cryptographic backend
    )
    # Derive the public key from the private key
    public_key = private_key.public_key()
    
    return private_key, public_key

def serialize_rsa_keys(private_key, public_key):
    """
    Serializes the RSA private and public keys to PEM format for storage or transmission.
    
    PEM format is a Base64 encoded message with header and footer to mark the beginning
    and end of the key material, making it suitable for text-based transmission.
    
    Args:
        private_key: The RSA private key to be serialized.
        public_key: The RSA public key to be serialized.
    
    Returns:
        A tuple containing the serialized private and public keys in PEM format.
    """
    # Serialize the private key to PEM format without encryption
    pem_private_key = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()  # Indicates no encryption
    )
    
    # Serialize the public key to PEM format
    pem_public_key = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return pem_private_key, pem_public_key

def sign_data_with_private_key(private_key, hashed):
    """
    Signs hashed data using the RSA private key.
    
    This function uses PSS (Probabilistic Signature Scheme) padding, which is recommended
    for new applications due to its security properties. The SHA-256 hash function is used
    for hashing the data, providing a strong level of integrity.
    
    Args:
        private_key: The RSA private key used for signing.
        hashed: The hashed data to be signed, typically the output of a secure hash function.
    
    Returns:
        The digital signature of the hashed data.
    """
    # Sign the hashed data with the private key using PSS padding and SHA-256
    signature = private_key.sign(
        hashed,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),  # Mask Generation Function
            salt_length=padding.PSS.MAX_LENGTH  # Maximum salt length
        ),
        hashes.SHA256()  # Hash function for signing
    )
    
    return signature

In [101]:
alice_private_key, alice_public_key = generate_rsa_keys()

pem_private_key, pem_public_key = serialize_rsa_keys(alice_private_key, alice_public_key)

signature = sign_data_with_private_key(alice_private_key, hashed.encode())

In [102]:
# Prepare the message for transmission
transmission_data = {
    'message': combined,
    'signature': signature,
    # 'encrypted_message': encrypted_message,  # If you chose to encrypt
}

# Simulate sending the data by outputting it
print(transmission_data)

{'message': 'afcd8d0d6d387866879d303001815b74:1710087696:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks', 'signature': b'0y\xb1\x83\xdf\x13\xd5\x18$\x02\xcaT8\x11\xa1O\xb4\x8f|fb\xe7b,\xdbj\xec\xe3nW\t?1\xa3\xa9<\x82\xb0^l\xe1\xf1/;e\x885\x7f\xa8\x08\x0f*\xe2&\xaa\x9dmZ\x821;\xdbzW+_8\xbfVo\x18\x9a\xd2\xdf\x81G|\x0c"\x80\x08\xd5\x00\xb7m\xb5[\x85\x81\xcb\xea\xa3jI\xdb\xb43R\xbc\x82\xd5P&RW\xaf5a\xc1Y\xda\n)Eh\xb7\x1a\xd2\x92\x0ftC\xc0\n\x9a\xb0\x1e\x10\x97Y\xc5\xe3\xb6\xfa\x91\xda\xa82\x0b\xa5\xc2\x0b\xd7r\xa5\xe3\xa5\t>x\xdc\xe7@]\x94\x00-\x89\xf4\xb1C9\xf7\x98\x89\xe8\xef\xe0><e\x81B\xd2\xa2\x87\x17\xfd\xc6\xfe`\xb9\xcd~\x08\x87RL\r7v\xf3\xbf\xc8\xdd\x005\x9c\x00Gk\xa9\xde7\x8c\x84\x82\xee\xf5g\xae\xdf\xfe\x17\xcf\x17\xb8;V\xb8\xe3N:\x0e\xacxz\x88\xe7P\x01\r%\x0f\xfe\xf6=\xeb\xda\xb8\xb6494dZXC\

In [103]:
def verify_signature(received_message, signature, public_key):
    """
    Verifies the signature of the received message using the sender's public key.
    """
    try:
        public_key.verify(
            signature,
            received_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False

In [104]:
def is_within_time_window(received_timestamp, window=300):
    current_time = int(time.time())
    return current_time - received_timestamp <= window

In [105]:
def is_nonce_unique(nonce, nonce_storage):
    if nonce in nonce_storage:
        return False  # Nonce is not unique, message should be rejected
    else:
        # Add the nonce to the storage to keep track of it for future messages
        nonce_storage.append(nonce)
        return True  # Nonce is unique, message can be processed

In [106]:
# Simulated received data that Bob recieves from Alice
received_data = transmission_data 

# Verify the signature of the received message
received_hashed_message = hashlib.sha256(received_data['message'].encode()).hexdigest()
verification_result = verify_signature(received_hashed_message.encode(), received_data['signature'], alice_public_key)

# Extract the timestamp from the received message
received_message = received_data['message']
nonce, received_timestamp_str, _ = received_message.split(':', 2)
received_timestamp = int(received_timestamp_str)

# Check if the received timestamp is within the allowed window
timestamp_verification_result = is_within_time_window(received_timestamp)

#Extract the nonce from the received message
nonce = received_data['message'].split(':', 1)[0] 
# Check if the nonce is unique
nonce_verification_result = is_nonce_unique(nonce, nonce_storage)

print(f"Signature verification result: {verification_result}")
print(f"Timestamp verification result: {timestamp_verification_result}")
print(f"Nonce verification result: {nonce_verification_result}")






Signature verification result: True
Timestamp verification result: True
Nonce verification result: True


In [107]:
received_message = received_data['message']

parts = received_message.split(':')
nonce = parts[0]
timestamp_str = parts[1]
actual_message = ':'.join(parts[2:])  # Join back the rest of the message in case there are semi-colons in it

# Convert the timestamp string to an integer
timestamp = int(timestamp_str)

# Convert the timestamp to a readable date format
dt_object = datetime.fromtimestamp(timestamp)
readable_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')

if verification_result:
    if timestamp_verification_result:
        if nonce_verification_result:
            print("The message is authentic, unique, and within the time window.")
            print(f"Nonce: {nonce}")
            print(f"Readable Timestamp: {readable_date}")
            print(f"Message: {actual_message}")
        else:
            print("The message failed verification: The nonce has already been used.")
    else:
        print("The message failed verification: The timestamp is outside the acceptable window.")
else:
    print("The message failed verification: The digital signature does not match.")


The message is authentic, unique, and within the time window.
Nonce: afcd8d0d6d387866879d303001815b74
Readable Timestamp: 2024-03-10 16:21:36
Message: The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


In [108]:
print(f"{nonce_storage}")

['afcd8d0d6d387866879d303001815b74']
